In [17]:
import os
import shutil

In [6]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Code/slurm_jobs'

In [12]:
indir = '../../../data/ABIDE2/RawData/'
folders = [f for f in os.listdir(indir) if f.startswith('ABIDEII')]
folders.sort()
folders

['ABIDEII-BNI_1',
 'ABIDEII-EMC_1',
 'ABIDEII-ETHZ_1',
 'ABIDEII-GU_1',
 'ABIDEII-IP_1',
 'ABIDEII-IU_1',
 'ABIDEII-KKI_1',
 'ABIDEII-KUL_3',
 'ABIDEII-NYU_1',
 'ABIDEII-NYU_2',
 'ABIDEII-OHSU_1',
 'ABIDEII-ONRC_2',
 'ABIDEII-SDSU_1',
 'ABIDEII-TCD_1',
 'ABIDEII-UCD_1',
 'ABIDEII-UCLA_1',
 'ABIDEII-UCLA_Long',
 'ABIDEII-UPSM_Long',
 'ABIDEII-USM_1']

In [18]:
subCount = list()
for folder in folders:
    nsubs = len([cont for cont in os.listdir(os.path.join(indir,folder)) if all((cont.startswith('sub-'),os.path.isdir(os.path.join(indir,folder,cont))))])
    subCount.append(nsubs)
    print(f'{folder} | {nsubs}'.rjust(30))
    

            ABIDEII-BNI_1 | 58
            ABIDEII-EMC_1 | 54
           ABIDEII-ETHZ_1 | 37
            ABIDEII-GU_1 | 106
             ABIDEII-IP_1 | 56
             ABIDEII-IU_1 | 40
           ABIDEII-KKI_1 | 211
            ABIDEII-KUL_3 | 28
            ABIDEII-NYU_1 | 78
            ABIDEII-NYU_2 | 27
           ABIDEII-OHSU_1 | 93
           ABIDEII-ONRC_2 | 59
           ABIDEII-SDSU_1 | 58
            ABIDEII-TCD_1 | 42
            ABIDEII-UCD_1 | 32
           ABIDEII-UCLA_1 | 32
        ABIDEII-UCLA_Long | 21
        ABIDEII-UPSM_Long | 17
            ABIDEII-USM_1 | 33


In [102]:
i = -1

In [285]:
i+=1
folder = folders[i]
n = subCount[i]
print(folder)
print(n)

IndexError: list index out of range

In [286]:
if not os.path.isdir(f'fmriprep_{folder}'):
    os.mkdir(f'fmriprep_{folder}')

In [287]:
text = '''#!/bin/bash

#SBATCH --job-name=job_fmriprep_{folder}
#SBATCH --output=./fmriprep_{folder}/output_fmriprep_{folder}_%a
#SBATCH --error=./fmriprep_{folder}/error_fmriprep_{folder}_%a

#SBATCH --ntasks=1
#SBATCH --time=10:00:00
#SBATCH --mem=32gb

#SBATCH --array=0-{n}

#rm -rf fmriprep_{folder}.slurm;nano fmriprep_{folder}.slurm
#sbatch fmriprep_{folder}.slurm
#$SLURM_ARRAY_TASK_ID


#rm -rf output_fmriprep_{folder}_%a
#rm -rf error_fmriprep_{folder}_%a

module load singularity/

#this_dir=$(pwd);echo $this_dir
cd $HOME

data_dir=$HOME'/data/ABIDE2/RawData/{folder}/'

sing_image=$HOME/fmriprep.simg

cd $data_dir
files=(sub*)
#cd $this_dir

cd $HOME

output_dir=$HOME'/data/ABIDE2/Derivatives/test/'

fs_lic=$HOME'/fs_licence.txt'

sub=${{files[$SLURM_ARRAY_TASK_ID]}} #;echo $sub

echo $HOME
echo $data_dir
echo $output_dir
echo $sub

#echo ${{files[$SLURM_ARRAY_TASK_ID]}}

singularity run --bind /data/aglinska/scratch:/scratch --cleanenv $sing_image $data_dir $output_dir participant --participant-label $sub --fs-no-reconall --task-id 'rest' --fs-license-file $fs_lic

'''

In [288]:
text = text.format(folder=folder,n=n)

In [289]:
with open(f'job_fmriprep_{folder}.slurm', 'w') as f:
    f.write(text)

In [290]:
bashCommand = f'sbatch job_fmriprep_{folder}.slurm'
print(bashCommand)

sbatch job_fmriprep_ABIDEII-USM_1.slurm


In [291]:
with open(f'command.sh', 'w') as f:
    f.write(bashCommand)

In [292]:
!chmod 777 command.sh

In [293]:
!./command.sh

Submitted batch job 164610


In [294]:
#!sbatch job_fmriprep_ABIDEII-ETHZ_1.slurm

In [297]:
#!squeue